In [1]:
!python3 --version
!pip freeze | grep qiskit
from qiskit.visualization import plot_histogram
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.circuit.library import TwoLocal, UGate, PauliEvolutionGate
from qiskit.quantum_info import Statevector, SparsePauliOp
import numpy as np
import matplotlib.pyplot as plt

Python 3.12.3
qiskit==1.1.0
qiskit-algorithms==0.3.0
qiskit-nature==0.7.2


The Fauseweh-Zhu paper [here](https://arxiv.org/pdf/2112.04276).

Variational Quantum Design Course from IBMQ Learning
- [Ansatz](https://learning.quantum.ibm.com/course/variational-algorithm-design/ansatze-and-variational-forms) include the NLocal, we use TwoLocal (but i already wrote this)
- [Optimisation](https://learning.quantum.ibm.com/course/variational-algorithm-design/ansatze-and-variational-forms) frameworks has also been discussed

- Also check out [Quantum Approximate Optimization Algorithm](https://learning.quantum.ibm.com/tutorial/quantum-approximate-optimization-algorithm)

In [2]:
chain_length = 4

num_layers = 1

shots = 2000

In [3]:
parameter_space_size = 2 * chain_length + 3 * chain_length * num_layers

# param_space = [Parameter(f'θ[{i}]') for i in range(parameter_space_size)]

param_space = range(1,parameter_space_size+1)

# param_space = [0]*parameter_space_size # initial guess

In [4]:
Ω = 2.5
periods = 2*np.pi/Ω
num_time_steps = 150
dt = periods / num_time_steps
A = 0

In [15]:
def ansatz_circuit_0(qc, param_space):
    print('Number of params:',parameter_space_size)
    # layer 0
    param_counter=0
    for i in range(chain_length):
        qc.rx(param_space[param_counter],i)
        param_counter=param_counter+1
        qc.rz(param_space[param_counter],i)
        param_counter=param_counter+1
    
def ansatz_circuit_1(qc, param_space):
    param_counter = 2 * chain_length
    for i in range(chain_length-1):
        qc.cx(i,i+1)
    qc.cx(-1,0)
    for i in range(chain_length):
        qc.rz(param_space[param_counter],i)
        param_counter=param_counter+1
        qc.rx(param_space[param_counter],i)
        param_counter=param_counter+1
        qc.rz(param_space[param_counter],i)
        param_counter=param_counter+1

def create_ansatz_circuit(qc, num_layers=num_layers, param_space=param_space):
    ansatz_circuit_0(qc, param_space)
    for i in range(num_layers):
        ansatz_circuit_1(qc, param_space)

def ansatz_circuit_ladder(qc, param_space):
    register_size = chain_length*2
    # layer 0
    param_counter = 0
    for i in range(register_size):
        qc.rx(param_space[param_counter],i)
        param_counter=param_counter+1
    for i in range(register_size):
        qc.rz(param_space[param_counter],i)
        param_counter=param_counter+1
    for i in range(1,register_size//2):
        place = 2 * i - 1
        qc.cx(place,place+1)

def hamiltonian(t, A=2, J=1, Ω=Ω):
    creator = ['I']*chain_length
    paulis = ['I','X','Y','Z']
    ham = [] # [('X',1.0)]
    for i in range(chain_length-1):
        for j in range(1,4):
            op = creator[:]
            op[i] = paulis[j]
            op[i+1] = paulis[j]
            ham.append([''.join(op), -J/4])
    for i in range(chain_length):
        op1, op2 = creator[:], creator[:]
        op1[i] = 'X'
        op2[i] = 'Y'
        ham.append([''.join(op1), A * np.cos(Ω*t)])
        ham.append([''.join(op2), A * np.sin(Ω*t)])
    ham = np.array(ham)
    return SparsePauliOp(ham[:,0], ham[:,1])

def unitary_time_evolution(ham, num_qbits=chain_length, time=num_time_steps*dt, dt=dt):#num_steps=num_time_steps):

    circuit = QuantumCircuit(num_qbits)
    
    for i in range(1, num_time_steps+1):
        circuit.append(PauliEvolutionGate(ham(i*dt), time=dt), range(num_qbits))

    # print('Unitary Evolution Circuit')
    # display(circuit.draw('mpl'))
    
    return circuit

def overlap(circuit1, circuit2): # < circuit1 | circuit2 >
    circuit_state1 = Statevector.from_instruction(circuit1)
    circuit_state2 = Statevector.from_instruction(circuit2)
    return circuit_state1.inner(circuit_state2)
    
def cost_function(circuit, unitary_time_evolution, computed_circuits: list, λ=5):
    summation = np.sum( [np.abs(overlap(circuit, i))**2 for i in computed_circuits] )
    evolved = circuit.compose(unitary_time_evolution)
    return np.abs( overlap(circuit, evolved) )**2 - λ * summation

def hamiltonian_linear(t, Δ=1, Ω=Ω):
    ham = SparsePauliOp(['Z','X'] , [-Δ/2, A/2/Ω*np.cos(Ω*t)])
    # plt.plot(t, A*np.cos(Ω*t)/2,'.')
    return ham

In [6]:
print(hamiltonian_linear(0,2))

SparsePauliOp(['Z', 'X'],
              coeffs=[-1.+0.j,  0.+0.j])


In [7]:
qc = QuantumCircuit(chain_length)

# qc.h(qc.qubits)

create_ansatz_circuit(qc, 2)

unitary = unitary_time_evolution(hamiltonian)

qc.compose(unitary, inplace=True)

# qc.measure_all()

# qc.draw('mpl')

# print(time_evolver(qc, hamiltonian(0)))

Number of params: 20


In [8]:
def linear(*initial_guess):
    test_qc = QuantumCircuit(1)
    # params = [Parameter(f'θ[{i}]') for i in range(3)]
    
    test_qc.u(*initial_guess, 0)

    unitary_timevo_circuit = unitary_time_evolution(hamiltonian_linear, 1)

    evolved = test_qc.compose(unitary_timevo_circuit)

    return overlap(test_qc, evolved)

    # display(test_qc.draw('mpl'))

A = 1

linear(0,0,0)    

(0.3180792608703056+0.9453370697203398j)

In [16]:
def main():
    test_qc = QuantumCircuit(chain_length)
    # params = [Parameter(f'θ[{i}]') for i in range(3)]

    create_ansatz_circuit(test_qc)

    # display(test_qc.draw('mpl'))

    unitary_timevo_circuit = unitary_time_evolution(hamiltonian)

    evolved = test_qc.compose(unitary_timevo_circuit)

    print(overlap(test_qc, evolved))

    # display(test_qc.draw('mpl'))

main()    

Number of params: 20
(-0.01730160281246523+0.1889759079819219j)


In [10]:
dt

0.016755160819145562